**POI分类构建去从聚字段，以类型总点数为调整因子，乘上去从聚字段，以和不进行去从聚调整的各项均为1形成对比，在此基础上，用去丛聚调整项来乘上权重项，以实现抽稀的功能**

# 去从聚

In [1]:
# 四叉树节点类
class QuadTreeNode:
    node_count = 0  # 用于生成唯一的格网ID
    
    def __init__(self, extent, max_points=1000, max_depth=5, depth=0):
        self.extent = extent  # 节点的空间范围
        self.children = []  # 子节点列表
        self.max_points = max_points  # 每个节点能包含的最大点数
        self.max_depth = max_depth  # 树的最大深度
        self.depth = depth  # 当前节点的深度
        self.points = []  # 此节点包含的POI点列表
        self.grid_id = QuadTreeNode.node_count  # 唯一的格网ID
        QuadTreeNode.node_count += 1
        
    # 分割当前节点为四个子节点
    def split(self):
        xmin, ymin, xmax, ymax = self.extent
        xmid, ymid = (xmin + xmax) / 2, (ymin + ymax) / 2
        
        # 定义四个子区域的空间范围
        extents = [
            (xmin, ymin, xmid, ymid),  # 西南
            (xmid, ymin, xmax, ymid),  # 东南
            (xmin, ymid, xmid, ymax),  # 西北
            (xmid, ymid, xmax, ymax)   # 东北
        ]
    
        # 为每个子区域创建一个新的四叉树节点
        for extent in extents:
            self.children.append(QuadTreeNode(extent, max_points=self.max_points, max_depth=self.max_depth, depth=self.depth + 1))
            
    # 将一个点插入四叉树中，并存储格网ID或节点引用
    def insert(self, point):
        if not self._contains(point):
            return False  # 点不在当前节点的范围内

        if len(self.points) < self.max_points or self.depth == self.max_depth:
            self.points.append(point)  # 如果节点未满，或达到最大深度，直接添加点
            return True

        if not self.children:
            self.split()  # 如果子节点不存在，先分割当前节点

        return any(child.insert(point) for child in self.children)
    
    # 检查点是否在当前节点的范围内
    def _contains(self, point):
        x, y = point
        xmin, ymin, xmax, ymax = self.extent
        return xmin <= x < xmax and ymin <= y < ymax

In [2]:
# 构建四叉树并为每个POI点分配格网ID
def build_quadtree_and_assign_grid_ids(poi_layer, root):
    with arcpy.da.UpdateCursor(poi_layer, ["SHAPE@XY", "GridID"]) as cursor:
        for row in cursor:
            point = row[0]
            if root.insert(point):
                # 如果点被插入到四叉树，更新其格网ID
                row[1] = find_grid_id(root, point)
                cursor.updateRow(row)

In [3]:
# 递归查找包含给定点的节点的格网ID
def find_grid_id(node, point):
    if node._contains(point):
        if not node.children or node.depth == node.max_depth:
            return node.grid_id
        for child in node.children:
            grid_id = find_grid_id(child, point)
            if grid_id is not None:
                return grid_id
    return None

In [4]:
# 计算每个格网内POI的数量
def calculate_grid_poi_counts(poi_layer):
    grid_poi_counts = {}
    with arcpy.da.SearchCursor(poi_layer, ["GridID"]) as cursor:
        for row in cursor:
            grid_id = row[0]
            grid_poi_counts[grid_id] = grid_poi_counts.get(grid_id, 0) + 1
    return grid_poi_counts

In [5]:
# 为每个POI点分配权重
def assign_weights_to_pois(poi_layer, grid_poi_counts):
    # 计算有POI的格网总数
    total_occupied_grids = len(grid_poi_counts)
    # 计算每个格网的权重
    grid_weight = 1.0 / total_occupied_grids if total_occupied_grids else 0
    
    with arcpy.da.UpdateCursor(poi_layer, ["GridID", "Decluster"]) as cursor:
        for row in cursor:
            grid_id = row[0]
            poi_count = grid_poi_counts.get(grid_id, 0)
            Decluster = grid_weight / poi_count if poi_count > 0 else 0
            row[1] = Decluster
            cursor.updateRow(row)

**定义参数**

In [6]:
# 根据研究区域边界矢量数据生成四叉树覆盖的空间范围
ring3 = r"E:\stu1\Projects\POI2023\POI2023.gdb\ring3"
boundary_extent = arcpy.Describe(ring3).extent
tree_extent = (boundary_extent.XMin, boundary_extent.YMin, boundary_extent.XMax, boundary_extent.YMax)

In [7]:
# 创建四叉树的根节点
max_points_per_node = 100
max_depth = 7
root = QuadTreeNode(tree_extent, max_points=max_points_per_node, max_depth=max_depth)

**遍历POI**

In [8]:
def process_poi_layer(poi_layer_name, poi_layer):
    print(f"开始处理图层: {poi_layer_name}")
    arcpy.AddField_management(poi_layer, "GridID", "LONG")
    arcpy.AddField_management(poi_layer, "Decluster", "FLOAT")
    build_quadtree_and_assign_grid_ids(poi_layer, root)
    grid_poi_counts = calculate_grid_poi_counts(poi_layer)
    assign_weights_to_pois(poi_layer, grid_poi_counts)
    print(f"图层处理完成: {poi_layer_name}")

In [9]:
# 列出所有待处理的 POI 图层名称及其对应的图层对象
poi_layers = {
    "shapefile_R": r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_R",
    "shapefile_A": r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_A",
    "shapefile_B": r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_B",
    "shapefile_M": r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_M",
    "shapefile_S": r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_s",
    "shapefile_G": r"E:\stu1\Projects\POI2023\POI2023.gdb\poi23\JOIN_G"
}

# 对每个图层执行处理，并加上进度说明
for poi_layer_name, poi_layer in poi_layers.items():
    process_poi_layer(poi_layer_name, poi_layer)

开始处理图层: shapefile_R
图层处理完成: shapefile_R
开始处理图层: shapefile_A
图层处理完成: shapefile_A
开始处理图层: shapefile_B
图层处理完成: shapefile_B
开始处理图层: shapefile_M
图层处理完成: shapefile_M
开始处理图层: shapefile_S
图层处理完成: shapefile_S
开始处理图层: shapefile_G
图层处理完成: shapefile_G


**乘上总点数**

In [10]:
def Adjust_Decluster(input_features, field_name, factor):
    """
    Adjusts the values in a specified field by multiplying them by a given factor.

    Args:
    - input_features: Path to the feature class or table.
    - field_name: Name of the field to be adjusted.
    - factor: The factor by which to multiply the field's values.
    """
    # Define the calculation expression
    calculation_expression = f"!{field_name}! * {factor}"

    # Perform the field calculation
    arcpy.CalculateField_management(input_features, field_name, calculation_expression, "PYTHON3")

    print(f"Values in '{field_name}' have been adjusted by a factor of {factor}.")

In [11]:
Adjust_Decluster("JOIN_A", "Decluster", "27797")

Values in 'Decluster' have been adjusted by a factor of 27797.


In [12]:
Adjust_Decluster("JOIN_B", "Decluster", "37901")

Values in 'Decluster' have been adjusted by a factor of 37901.


In [13]:
Adjust_Decluster("JOIN_G", "Decluster", "829")

Values in 'Decluster' have been adjusted by a factor of 829.


In [14]:
Adjust_Decluster("JOIN_M", "Decluster", "21495")

Values in 'Decluster' have been adjusted by a factor of 21495.


In [15]:
Adjust_Decluster("JOIN_S", "Decluster", "55")

Values in 'Decluster' have been adjusted by a factor of 55.


In [19]:
Adjust_Decluster("JOIN_R", "Decluster", "7172")

Values in 'Decluster' have been adjusted by a factor of 7172.


# 计算调整后的权重

In [16]:
def Adjust_Weight(input_features, new_field_name, weight_field, decluster_field):
    """
    Creates a new field and populates it with the product of two existing fields.

    Args:
    - input_features: Path to the feature class or table.
    - new_field_name: Name of the new field to be added and calculated.
    - weight_field: Name of the weight field to be used in the calculation.
    - decluster_field: Name of the decluster field to be used in the calculation.
    """
    # Check if the new field already exists, and add it if not
    field_list = [field.name for field in arcpy.ListFields(input_features)]
    if new_field_name not in field_list:
        arcpy.AddField_management(input_features, new_field_name, "DOUBLE")
        print(f"Field '{new_field_name}' added to {input_features}.")
    else:
        print(f"Field '{new_field_name}' already exists.")

    # Define the calculation expression
    calculation_expression = f"!{weight_field}! * !{decluster_field}!"

    # Perform the field calculation
    arcpy.CalculateField_management(input_features, new_field_name, calculation_expression, "PYTHON3")

    print(f"Field '{new_field_name}' has been populated with calculated values.")

In [18]:
Adjust_Weight("JOIN_A", "DE_Weight", "Weight", "Decluster")

Field 'DE_Weight' added to JOIN_A.
Field 'DE_Weight' has been populated with calculated values.


In [20]:
Adjust_Weight("JOIN_B", "DE_Weight", "Weight", "Decluster")

Field 'DE_Weight' added to JOIN_B.
Field 'DE_Weight' has been populated with calculated values.


In [21]:
Adjust_Weight("JOIN_G", "DE_Weight", "Weight", "Decluster")

Field 'DE_Weight' added to JOIN_G.
Field 'DE_Weight' has been populated with calculated values.


In [22]:
Adjust_Weight("JOIN_M", "DE_Weight", "Weight", "Decluster")

Field 'DE_Weight' added to JOIN_M.
Field 'DE_Weight' has been populated with calculated values.


In [23]:
Adjust_Weight("JOIN_S", "DE_Weight", "Weight", "Decluster")

Field 'DE_Weight' added to JOIN_S.
Field 'DE_Weight' has been populated with calculated values.


In [24]:
Adjust_Weight("JOIN_R", "DE_Weight", "Weight", "Decluster")

Field 'DE_Weight' added to JOIN_R.
Field 'DE_Weight' has been populated with calculated values.


# 用调整后的权重计算频率密度

In [26]:
def calculate_and_update_frequency_density(input_poi_layer, weight_field, block_id_field, block_layer, frequency_density_field):
    # 初始化用于汇总每个街区加权POI点数的字典
    block_weights = {}
    total_weight = 0

    # 第1步：按街区代码汇总POI的权重
    with arcpy.da.SearchCursor(input_poi_layer, [block_id_field, weight_field]) as cursor:
        for row in cursor:
            block_id, weight = row
            block_weights[block_id] = block_weights.get(block_id, 0) + weight
            total_weight += weight

    # 第2步：计算每个街区的频率密度
    block_freq_density = {block: weight / total_weight for block, weight in block_weights.items()}

    # 第3步：在街区要素类中创建频率密度字段（如果尚不存在）
    if len(arcpy.ListFields(block_layer, frequency_density_field)) == 0:
        arcpy.AddField_management(block_layer, frequency_density_field, "DOUBLE")

    # 更新街区要素类中的频率密度字段
    with arcpy.da.UpdateCursor(block_layer, [block_id_field, frequency_density_field]) as cursor:
        for row in cursor:
            block_id = row[0]
            if block_id in block_freq_density:
                row[1] = block_freq_density[block_id]  # 更新频率密度
            else:
                row[1] = 0  # 对于没有POI点的街区，频率密度设置为0
            cursor.updateRow(row)

    print("街区要素类中的频率密度字段更新完成。")

In [27]:
calculate_and_update_frequency_density('JOIN_A', 'DE_Weight', 'Block_ID', 'blocks_initial', 'DE_FD_A')

街区要素类中的频率密度字段更新完成。


In [28]:
calculate_and_update_frequency_density('JOIN_B', 'DE_Weight', 'Block_ID', 'blocks_initial', 'DE_FD_B')

街区要素类中的频率密度字段更新完成。


In [29]:
calculate_and_update_frequency_density('JOIN_R', 'DE_Weight', 'Block_ID', 'blocks_initial', 'DE_FD_R')

街区要素类中的频率密度字段更新完成。


In [30]:
calculate_and_update_frequency_density('JOIN_M', 'DE_Weight', 'Block_ID', 'blocks_initial', 'DE_FD_M')

街区要素类中的频率密度字段更新完成。


In [31]:
calculate_and_update_frequency_density('JOIN_G', 'DE_Weight', 'Block_ID', 'blocks_initial', 'DE_FD_G')

街区要素类中的频率密度字段更新完成。


In [32]:
calculate_and_update_frequency_density('JOIN_S', 'DE_Weight', 'Block_ID', 'blocks_initial', 'DE_FD_S')

街区要素类中的频率密度字段更新完成。


# 用调整后的权重计算核密度

In [35]:
import arcpy
from arcpy.sa import *

def perform_kernel_density_and_update_blocks(input_features, population_field, search_radius, cell_size, output_raster_name, block_feature_class, zone_field, new_field_name):
    """
    执行核密度分析，并将结果按街区汇总，最后更新到街区要素类的新字段中。

    参数:
    - input_features: 输入的POI图层路径。
    - population_field: 使用的权重字段名。
    - search_radius: 核密度分析的搜索半径。
    - cell_size: 核密度分析的栅格单元大小。
    - output_raster_name: 输出核密度栅格图层的名称。
    - block_feature_class: 街区要素类路径。
    - zone_field: 街区要素类中用于标识每个街区的字段名。
    - new_field_name: 更新到街区要素类中的新字段名。
    """
    # 确保空间分析扩展可用
    arcpy.CheckOutExtension("Spatial")
    
    # 核密度分析
    kd_result = KernelDensity(input_features, population_field, cell_size, search_radius, area_unit_scale_factor="SQUARE_METERS")
    kd_result.save(output_raster_name)
    print(f"{output_raster_name} 核密度分析完成")
    
    # 执行区域统计作为表格
    out_table = output_raster_name + "_ZonalStats"
    ZonalStatisticsAsTable(block_feature_class, zone_field, output_raster_name, out_table, "DATA", "SUM")
    
    # 使用 JoinField 将统计表格的 SUM 字段添加到街区要素类的属性表中
    arcpy.JoinField_management(block_feature_class, zone_field, out_table, "OBJECTID", ["SUM"])
    
    # 添加新字段（如果尚未存在）
    if len(arcpy.ListFields(block_feature_class, new_field_name)) == 0:
        arcpy.AddField_management(block_feature_class, new_field_name, "DOUBLE")
    
    # 复制 SUM 字段的值到新字段，并删除 SUM 字段
    with arcpy.da.UpdateCursor(block_feature_class, ["SUM", new_field_name]) as cursor:
        for row in cursor:
            row[1] = row[0]  # 将 SUM 字段的值复制到新字段
            cursor.updateRow(row)
    arcpy.DeleteField_management(block_feature_class, "SUM")
    print(f"{new_field_name} 字段已更新")


In [52]:
perform_kernel_density_and_update_blocks(
    input_features="JOIN_A_Project",
    population_field="DE_Weight",
    search_radius=2000,
    cell_size=10,
    output_raster_name="DE_KernelDensity_A",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="DE_KD_A"
)

DE_KernelDensity_A 核密度分析完成
DE_KD_A 字段已更新


In [53]:
perform_kernel_density_and_update_blocks(
    input_features="JOIN_B_Project",
    population_field="DE_Weight",
    search_radius=1500,
    cell_size=10,
    output_raster_name="DE_KernelDensity_B",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="DE_KD_B"
)


DE_KernelDensity_B 核密度分析完成
DE_KD_B 字段已更新


In [54]:
perform_kernel_density_and_update_blocks(
    input_features="JOIN_G_Project",
    population_field="DE_Weight",
    search_radius=2000,
    cell_size=10,
    output_raster_name="DE_KernelDensity_G",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="DE_KD_G"
)


DE_KernelDensity_G 核密度分析完成
DE_KD_G 字段已更新


In [55]:
perform_kernel_density_and_update_blocks(
    input_features="JOIN_M_Project",
    population_field="DE_Weight",
    search_radius=1500,
    cell_size=10,
    output_raster_name="DE_KernelDensity_M",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="DE_KD_M"
)


DE_KernelDensity_M 核密度分析完成
DE_KD_M 字段已更新


In [56]:
perform_kernel_density_and_update_blocks(
    input_features="JOIN_S_Project",
    population_field="DE_Weight",
    search_radius=3000,
    cell_size=10,
    output_raster_name="DE_KernelDensity_S",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="DE_KD_S"
)


DE_KernelDensity_S 核密度分析完成
DE_KD_S 字段已更新


In [57]:
perform_kernel_density_and_update_blocks(
    input_features="JOIN_R_Project",
    population_field="DE_Weight",
    search_radius=1000,
    cell_size=10,
    output_raster_name="DE_KernelDensity_R",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="DE_KD_R"
)


DE_KernelDensity_R 核密度分析完成
DE_KD_R 字段已更新


# 用调整后的权重计算网络约束的核密度

In [1]:
import arcpy
from arcpy.sa import *

def KDB2blocks(raster_name, block_feature_class, zone_field, new_field_name):
    """
    执行核密度分析，并将结果按街区汇总，最后更新到街区要素类的新字段中。

    参数:
    - raster_name: 核密度栅格图层的名称。
    - block_feature_class: 街区要素类路径。
    - zone_field: 街区要素类中用于标识每个街区的字段名。
    - new_field_name: 更新到街区要素类中的新字段名。
    """
    
    # 执行区域统计作为表格
    out_table = raster_name + "_ZonalStats_BARR"
    ZonalStatisticsAsTable(block_feature_class, zone_field, raster_name, out_table, "DATA", "SUM")
    
    # 使用 JoinField 将统计表格的 SUM 字段添加到街区要素类的属性表中
    arcpy.JoinField_management(block_feature_class, zone_field, out_table, "OBJECTID", ["SUM"])
    
    # 添加新字段（如果尚未存在）
    if len(arcpy.ListFields(block_feature_class, new_field_name)) == 0:
        arcpy.AddField_management(block_feature_class, new_field_name, "DOUBLE")
    
    # 复制 SUM 字段的值到新字段，并删除 SUM 字段
    with arcpy.da.UpdateCursor(block_feature_class, ["SUM", new_field_name]) as cursor:
        for row in cursor:
            row[1] = row[0]  # 将 SUM 字段的值复制到新字段
            cursor.updateRow(row)
    arcpy.DeleteField_management(block_feature_class, "SUM")
    print(f"{new_field_name} 字段已更新")

In [2]:
# Copy Python command
out_raster = arcpy.sa.KernelDensity(r"E:\stu1\Projects\POI2023\POI2023.gdb\JOIN_A_Project", 
                        "DE_Weight", 10, 2000, "SQUARE_METERS", "DENSITIES", 
                        "PLANAR", "高架及快速路_Centerline"); out_raster.save(r"E:\stu1\Projects\POI2023\POI2023.gdb\DE_KDB_A")

In [3]:
KDB2blocks(
    raster_name="DE_KDB_A",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="DE_KDB_A"
)

DE_KDB_A 字段已更新


In [14]:
out_raster = arcpy.sa.KernelDensity(r"E:\stu1\Projects\POI2023\POI2023.gdb\JOIN_B_Project", 
                        "DE_Weight", 10, 1500, "SQUARE_METERS", "DENSITIES", 
                        "PLANAR", "高架及快速路_Centerline"); out_raster.save(r"E:\stu1\Projects\POI2023\POI2023.gdb\DE_KDB_B")

In [15]:
KDB2blocks(
    raster_name="DE_KDB_B",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="DE_KDB_B"
)

DE_KDB_B 字段已更新


In [16]:
out_raster = arcpy.sa.KernelDensity(r"E:\stu1\Projects\POI2023\POI2023.gdb\JOIN_G_Project", 
                        "DE_Weight", 10, 2000, "SQUARE_METERS", "DENSITIES", 
                        "PLANAR", "高架及快速路_Centerline"); out_raster.save(r"E:\stu1\Projects\POI2023\POI2023.gdb\DE_KDB_G")

In [17]:
KDB2blocks(
    raster_name="DE_KDB_G",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="DE_KDB_G"
)

DE_KDB_G 字段已更新


In [18]:
out_raster = arcpy.sa.KernelDensity(r"E:\stu1\Projects\POI2023\POI2023.gdb\JOIN_M_Project", 
                        "DE_Weight", 10, 1500, "SQUARE_METERS", "DENSITIES", 
                        "PLANAR", "高架及快速路_Centerline"); out_raster.save(r"E:\stu1\Projects\POI2023\POI2023.gdb\DE_KDB_M")

In [19]:
KDB2blocks(
    raster_name="DE_KDB_M",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="DE_KDB_M"
)

DE_KDB_M 字段已更新


In [20]:
out_raster = arcpy.sa.KernelDensity(r"E:\stu1\Projects\POI2023\POI2023.gdb\JOIN_S_Project", 
                        "DE_Weight", 10, 3000, "SQUARE_METERS", "DENSITIES", 
                        "PLANAR", "高架及快速路_Centerline"); out_raster.save(r"E:\stu1\Projects\POI2023\POI2023.gdb\DE_KDB_S")

In [21]:
KDB2blocks(
    raster_name="DE_KDB_S",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="DE_KDB_S"
)

DE_KDB_S 字段已更新


In [22]:
out_raster = arcpy.sa.KernelDensity(r"E:\stu1\Projects\POI2023\POI2023.gdb\JOIN_R_Project", 
                        "DE_Weight", 10, 1000, "SQUARE_METERS", "DENSITIES", 
                        "PLANAR", "高架及快速路_Centerline"); out_raster.save(r"E:\stu1\Projects\POI2023\POI2023.gdb\DE_KDB_R")

In [23]:
KDB2blocks(
    raster_name="DE_KDB_R",
    block_feature_class="blocks_initial",
    zone_field="Block_ID",
    new_field_name="DE_KDB_R"
)

DE_KDB_R 字段已更新
